In [1]:
!pip install hyperopt

In [11]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.layers.pooling import MaxPooling2D as pooling

from sklearn.metrics import accuracy_score
from skimage import color

%load_ext tensorboard

import matplotlib.pyplot as plt
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle("data/train.p")
test = pd.read_pickle("data/test.p")

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'], )

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, 
            y_train,
            bath_size = params_fit.get('bath_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_train, y_train)),
            callbacks = [tensorboard_callback]
            )
  return model

def predict(trained_model, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_pred, y_test_norm)

def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [7]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),

        Dense(1024,activation='relu'),
        Dropout(0.3),

        Dense(1024,activation='relu'),
        Dropout(0.3),

        Dense(num_classes, activation='softmax'),
  ])

train_and_predict( get_cnn_v5(input_shape, num_classes) )

Epoch 1/5
1088/1088 [==============================] - 9s 9ms/step - loss: 1.8513 - accuracy: 0.4652 - val_loss: 0.5279 - val_accuracy: 0.8110
Epoch 2/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.5981 - accuracy: 0.8128 - val_loss: 0.1959 - val_accuracy: 0.9408
Epoch 3/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.3991 - accuracy: 0.8774 - val_loss: 0.1322 - val_accuracy: 0.9577
Epoch 4/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.3417 - accuracy: 0.8990 - val_loss: 0.0959 - val_accuracy: 0.9701
Epoch 5/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.3136 - accuracy: 0.9083 - val_loss: 0.1077 - val_accuracy: 0.9684


0.935374149659864

In [26]:
def get_model(params):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),

        Dense(1024,activation='relu'),
        Dropout(params['dropout_dense_one']),

        Dense(1024,activation='relu'),
        Dropout(params['dropout_dense_two']),

        Dense(num_classes, activation='softmax'),
  ])

train_and_predict( get_cnn_v5(input_shape, num_classes) )

Epoch 1/5
1088/1088 [==============================] - 9s 9ms/step - loss: 2.0365 - accuracy: 0.4135 - val_loss: 0.6983 - val_accuracy: 0.8009
Epoch 2/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.6943 - accuracy: 0.7763 - val_loss: 0.2538 - val_accuracy: 0.9119
Epoch 3/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.4475 - accuracy: 0.8648 - val_loss: 0.1425 - val_accuracy: 0.9577
Epoch 4/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.3456 - accuracy: 0.9003 - val_loss: 0.1266 - val_accuracy: 0.9616
Epoch 5/5
1088/1088 [==============================] - 9s 8ms/step - loss: 0.3211 - accuracy: 0.9092 - val_loss: 0.1078 - val_accuracy: 0.9676


0.9369614512471656

In [0]:
def get_model_2(params):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

       Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),

        Dense(1024,activation='relu'),
        Dropout(params['dropout_dense_one']),

        Dense(1024,activation='relu'),
        Dropout(params['dropout_dense_two']),

        Dense(num_classes, activation='softmax'),
  ])

#train_and_predict( get_cnn_v5(input_shape, num_classes) )

In [0]:
def func_obj(params):
  model = get_model_2(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'], )

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, 
            y_train,
            bath_size = int(params.get('bath_size', 128)),
            epochs = 10,
            verbose = 0
            )
  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy = {}'.format(accuracy))
  return {'loss' : -accuracy, 'status': STATUS_OK, 'model': model}

In [36]:
space = {
    'bath_size': hp.quniform('bath_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.2, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.2, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.2, 0.5),

    'dropout_dense_one': hp.uniform('dropout_dense_one', 0.3, 0.7),
    'dropout_dense_two': hp.uniform('dropout_dense_two', 0.3, 0.7),
}

best=fmin(
    func_obj,
    space,
    tpe.suggest,
    50,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.8007 - accuracy: 0.6875

 20/138 [===>..........................]
 - ETA: 0s - loss: 0.5267 - accuracy: 0.8516

 36/138 [======>.......................]
 - ETA: 0s - loss: 0.3867 - accuracy: 0.8767

 53/138 [==========>...................]
 - ETA: 0s - loss: 0.4464 - accuracy: 0.8626

 70/138 [==============>...............]
 - ETA: 0s - loss: 0.4179 - accuracy: 0.8862

 87/138 [=================>............]
 - ETA: 0s - loss: 0.3666 - accuracy: 0.9001


In [37]:
best

{'bath_size': 150.0,
 'dropout_cnn_block_one': 0.4365722992292878,
 'dropout_cnn_block_three': 0.40216885889786647,
 'dropout_cnn_block_two': 0.20087914319102196,
 'dropout_dense_one': 0.6674004323896826,
 'dropout_dense_two': 0.4403858510731814}